# HK Stock Screening (In Trade) v0

In this notebook we try to achieve
- Given a list of Ticker, compare Volumn from 2 day ago til today
- Introduce Nasdaq API
- Screen HK Common Stock with previous day Volume


Timeline as follow:

|| T0 | T1 | T2 | T3 |
| --- | --- | --- | --- | --- |
| Expected trend | x | Volume Rise | Volume Drop | Price Rise |
| Action | x | Create Potential Stocks List | **Enter Market near day end** | |

Import Library

In [ ]:
import numpy as np
import pandas as pd
import requests
import xlsxwriter
import math
from scipy import stats
import nasdaqdatalink
from datetime import date

Import Token

In [ ]:
from secrets import NASDAQ_API_TOKEN
nasdaqdatalink.ApiConfig.api_key = NASDAQ_API_TOKEN
SEND_TELEGRAM = False

Nasdaq Data Link API test call

In [ ]:
# data = nasdaqdatalink.get('HKEX/00213',
#                           column_index=10,
#                           rows=30,
#                           order='asc',
#                          )

# data['Share Volume (000)'].mean()


## Preparation

Fetch ticker of "today" watch list (which is generated in last trading day).

In which will also define "T0 date" of this screening.

In [ ]:
data = nasdaqdatalink.get('HKEX/06823',
                          rows=2,
                          order='asc',
                         )
data
# str(data.iloc[0].name.date()).replace("-", "")

In [ ]:
lastTradingDate = str(data.iloc[0].name.date()).replace("-", "")
lastTradingDate

In [ ]:
potentialStocks = pd.read_csv(f'../export/{lastTradingDate}/yau4muk6man4_strategy_HK_{lastTradingDate}_volume_rise.csv')
potentialStocks

## Data Fetching

Create main DataFrame, including volume of today, previous trading date and the day before.

Data imported from IEX Cloud API.

In [ ]:
%%time

finalDataFrameColumns = [
    'Ticker',
    'Price',
    '30 day Average Volume',
    'T0 Date',
    'T0 Volume',
    'T1 Volume',
    'T1 Volume Ratio',
    'T2 Volume',
    'T2 Volume Ratio',
]

finalDataFrame = pd.DataFrame(columns = finalDataFrameColumns)

for symbol in potentialStocks['Ticker']:
    data = nasdaqdatalink.get(f'HKEX/{symbol:05}',
                              rows=3,
                              order='asc'
                             )
#     print(data)
    try:
        latestPrice = data.iloc[-1]['Nominal Price']
    except (KeyError, IndexError):
        latestPrice = np.NaN
        
    try:
        t0Volume = data.iloc[0]['Share Volume (000)']
    except (KeyError, IndexError):
        try:
            t0Volume = data.iloc[0]['Share Volume (\'000)']
        except (KeyError, IndexError):
            t0Volume = np.NaN
    
    try:
        t1Volume = data.iloc[1]['Share Volume (000)']
    except (KeyError, IndexError):
        try:
            t1Volume = data.iloc[1]['Share Volume (\'000)']
        except (KeyError, IndexError):
            t1Volume = np.NaN
            
    try:
        t2Volume = data.iloc[2]['Share Volume (000)']
    except (KeyError, IndexError):
        try:
            t2Volume = data.iloc[2]['Share Volume (\'000)']
        except (KeyError, IndexError):
            t2Volume = np.NaN
        
        

    finalDataFrame = finalDataFrame.append(
        pd.Series(
            [
                f'{symbol:05}',
                latestPrice,
                'N/A',
                str(data.iloc[0].name.date()),
                t0Volume * 1000,
                t1Volume * 1000,
                'N/A',
                t2Volume * 1000,
                'N/A',
            ],
            index = finalDataFrameColumns
        ),
        ignore_index = True
    )

In [ ]:
finalDataFrame

## Data Cleaning

Clean out data with None from API

In [ ]:
finalDataFrame[finalDataFrame.isnull().any(axis = 1)]
# finalDataFrame[finalDataFrame['Price'].isnull()]

In [ ]:
finalDataFrame = finalDataFrame.dropna()
finalDataFrame

Drop data with 0 Volume

In [ ]:
finalDataFrame[
    (finalDataFrame['T2 Volume'] == 0)
]

In [ ]:
finalDataFrame.drop(
    finalDataFrame[
        (finalDataFrame['T2 Volume'] == 0)
    ].index,
    inplace = True
)
finalDataFrame

## Calculation

Calculate Volume change Ratio

In [ ]:
for row in finalDataFrame.index:
    finalDataFrame.loc[row, 'T1 Volume Ratio'] = finalDataFrame.loc[row, 'T1 Volume'] / finalDataFrame.loc[row, 'T0 Volume']
    finalDataFrame.loc[row, 'T2 Volume Ratio'] = finalDataFrame.loc[row, 'T2 Volume'] / finalDataFrame.loc[row, 'T1 Volume']
    
finalDataFrame

## Screening

Screening criteria are as follow:

- T2 Volume is below 25% of T1

In [ ]:
t2traget = float(0.25)

finalDataFrame = finalDataFrame[
    (finalDataFrame['T2 Volume Ratio'] < t2traget)
]
finalDataFrame.sort_values(
    'Ticker',
    ascending = True,
    inplace = True
)
finalDataFrame.reset_index(inplace = True, drop = True)

finalDataFrame

## Add 30 days average

As Nasdaq API did not return 30-days-average, we have to calculate ourselves.

In [ ]:
for row in finalDataFrame.index:
    symbol = finalDataFrame.loc[row, 'Ticker']
    averageVolume = nasdaqdatalink.get(f'HKEX/{symbol}',
                                       column_index=10,
                                       rows=30,
                                       order='asc',
                                      ).mean() * 1000
    finalDataFrame.loc[row, '30 day Average Volume'] = float(averageVolume)

finalDataFrame

## Data Export

In [ ]:
from datetime import date
import os

today = date.today().strftime("%Y%m%d")
folderName = f'../export/{today}'
fileName = f'{folderName}/yau4muk6man4_strategy_HK_{today}_volume_drop'
folderName = f'../export/{today}/'
csvFileName = f'{fileName}.csv'
xlsxFileName = f'{fileName}.xlsx'
sheetName = f'{today} Volume Drop'

In [ ]:
try:
    os.mkdir(folderName)
    print("Directory " , folderName ,  " Created ") 
except FileExistsError:
    print("Directory " , folderName ,  " already exists")

Export to xlsx

In [ ]:
writer = pd.ExcelWriter(xlsxFileName, engine = 'xlsxwriter')
finalDataFrame.to_excel(writer, sheet_name = sheetName, index = False)

In [ ]:
backgroundColor = '#0a0a23'
fontColor = '#ffffff'

stringTemplate = writer.book.add_format(
        {
            'font_color': fontColor,
            'bg_color': backgroundColor,
            'border': 1
        }
    )

dollarTemplate = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': fontColor,
            'bg_color': backgroundColor,
            'border': 1
        }
    )

integerTemplate = writer.book.add_format(
        {
            'num_format':'#,###',
            'font_color': fontColor,
            'bg_color': backgroundColor,
            'border': 1
        }
    )

floatTemplate = writer.book.add_format(
        {
            'num_format':'0.0',
            'font_color': fontColor,
            'bg_color': backgroundColor,
            'border': 1
        }
    )

percentTemplate = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': fontColor,
            'bg_color': backgroundColor,
            'border': 1
        }
    )

In [ ]:
columnFormats = {
    'A': ['Ticker', stringTemplate],
    'B': ['Price', dollarTemplate],
    'C': ['30 day Average Volume', integerTemplate],
    'D': ['T0 Date', stringTemplate],
    'E': ['T0 Volume', integerTemplate],
    'F': ['T1 Volume', integerTemplate],
    'G': ['T1 Volume Ratio', percentTemplate],
    'H': ['T2 Volume', integerTemplate],
    'I': ['T2 Volume Ratio', percentTemplate],
}

for column in columnFormats.keys():
    writer.sheets[sheetName].set_column(
        f'{column}:{column}',
        max(len(columnFormats[column][0]), 10),
        columnFormats[column][1]
    )
    writer.sheets[sheetName].write(
        f'{column}1',
        columnFormats[column][0],
        columnFormats[column][1]
    )

In [ ]:
writer.save()

## Telegram

In [ ]:
import telegram
from secrets import TELEGRAM_CHAT_ID
from secrets import TELEGRAM_API_TOKEN

In [ ]:
telegramBot = telegram.Bot(TELEGRAM_API_TOKEN)
telegramBot

In [ ]:
txt = f'''
⬇️⬇️⬇️ {today} Volume Drop ⬇️⬇️⬇️

This is report of stock that have drop in volume during current trading period ({today}).
We have {len(potentialStocks.index)} potential stock(s), in which {len(finalDataFrame.index)} have a low volume now.

Drop Ticker:
{','.join(finalDataFrame['Ticker'])}

Detail as in follow xlsx.
'''

output = open(xlsxFileName, 'rb')

In [ ]:
if SEND_TELEGRAM:
    telegramBot.send_message(TELEGRAM_CHAT_ID, txt)
    telegramBot.send_document(TELEGRAM_CHAT_ID, output)